In [1]:
# importando librerias
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import collections
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost
import math as mt
import warnings

# Ploting styles
# styles: 'fivethirtyeight', 'classic', 'ggplot', 'seaborn-notebook'
# styles: 'seaborn-poster', 'bmh', 'grayscale', 'seaborn-whitegrid'
matplotlib.style.use('bmh')
%matplotlib inline
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#print(plt.style.available)
warnings.filterwarnings("ignore")

In [4]:
#propios utilitarios
from utils import *
data_train = pd.read_csv('data/01dataBaseTrainTrxRec.csv', **set_parameter_csv)
data_train.head()

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto
0,2017-01-29 00:00:00,7649,138.0,43629,1,176.0,1,0.014072
1,2016-12-01 00:00:00,24604,75.0,4326,0,81.0,1,0.001667
2,2017-06-01 00:00:00,15289,75.0,4326,0,81.0,1,0.000127
3,2017-09-22 00:00:00,5190,110.0,59776,1,156.0,1,0.001167
4,2017-05-05 00:00:00,16635,75.0,31043,0,81.0,1,0.047386


In [6]:
data_train.sort_values(by=['codCliente','codEstab']).head(20)

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto
25041,2017-05-24 00:00:00,1,105.0,39,1,147.0,1,0.084590
522851,2017-03-07 00:00:00,1,105.0,39,1,147.0,1,0.098481
743922,2017-08-20 00:00:00,1,105.0,39,1,147.0,1,0.018126
1272804,2017-05-13 00:00:00,1,105.0,39,1,147.0,1,0.018183
816568,2016-12-22 00:00:00,1,72.0,1194,1,147.0,1,0.031735
512607,2017-08-03 00:00:00,1,66.0,1911,1,136.0,1,0.119995
867506,2017-08-06 00:00:00,1,NaN,2399,0,NaN,1,0.009543
1331328,2016-11-20 00:00:00,1,110.0,2425,1,170.0,1,0.012931
1478724,2017-06-20 00:00:00,1,110.0,2425,1,170.0,1,0.011858
400101,2017-08-26 00:00:00,1,50.0,3843,1,170.0,1,0.056412


In [8]:
pd.DataFrame(data.groupby(['codCliente', 'codEstab'])['ratingMonto'].mean())

ratingMonto
codCliente codEstab             
1          39           0.054845
           1194         0.031735
           1911         0.119995
           2399         0.009543
           2425         0.012395
           3843         0.048695
           4311         0.008188
           4616         0.018493
           6314         0.006296
           18153        0.008414
           19848        0.007623
           21136        0.007002
           21836        0.006494
           25407        0.036648
           26067        0.010729
           28921        0.001638
           29255        0.011745
           40261        0.011774
           43436        0.018578
           44974        0.006607
           45304        0.030437
           72869        0.033881
           81333        0.014682
2          37           0.016731
           663          0.010449
           826          0.002587
           1268         0.015152
           1751         0.009911
           1837         0.006383
           1986         0.004939
...                          ...
29998      31048        0.041565
           31965        0.040342
           43791        0.058680
           43969        0.014670
           47196        0.012225
           47214        0.009780
           47255        0.009169
           47278        0.066015
           47302        0.009780
29999      17824        0.011070
           31082        0.092251
           50137        0.092251
           51966        0.073801
           55247        0.092251
30000      514          0.026961
           1036         0.049020
           6045         0.009191
           17908        0.009191
           18786        0.018382
           29706        0.045956
           36463        0.018382
           39409        0.057598
           43281        0.042279
           49896        0.030025
           51084        0.034926
           59390        0.021446
           62569        0.012255
           71298        0.036765
           74702        0.139093
           75714        0.071691

[901967 rows x 1 columns]

In [17]:
data_train.describe(include='all')

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto
count,1591617,1.591617e+06,1.498931e+06,1.591617e+06,1.591617e+06,1.498931e+06,1.591617e+06,1.591617e+06
unique,365,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2017-07-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,7303,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.334583e+04,1.048248e+02,2.453260e+04,8.084175e-01,1.747467e+02,1.072021e+00,1.318383e-02
std,NaN,8.493548e+03,3.920585e+01,1.990371e+04,3.935464e-01,6.856421e+01,3.465646e-01,3.236438e-02
min,NaN,1.000000e+00,2.000000e+00,1.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,2.984905e-06
25%,NaN,6.035000e+03,7.500000e+01,4.339000e+03,1.000000e+00,1.480000e+02,1.000000e+00,1.598366e-03
50%,NaN,1.258100e+04,1.080000e+02,2.266500e+04,1.000000e+00,1.650000e+02,1.000000e+00,4.199563e-03
75%,NaN,2.046200e+04,1.120000e+02,3.673100e+04,1.000000e+00,1.750000e+02,1.000000e+00,1.152572e-02
